In [1]:
import numpy as np
import pandas as pd
from src.dataloader_ import *
from src.network_ import *
from src.utils import *

import os
import sys
import gc
import pickle
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer, KNNImputer

train_series_dir = "../../inputs/series_train.parquet/"
test_series_dir = "../../inputs/series_test.parquet/"

data_dic_path = "../../inputs/data_dictionary.csv"
sample_submission_path = "../../inputs/sample_submission.csv"
train_path = "../../inputs/train.csv"
test_path = "../../inputs/test.csv"

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
sample_submission = pd.read_csv(sample_submission_path)
data_dic = pd.read_csv(data_dic_path)

import os
import random

import numpy as np
import torch


def seed_torch(seed=1029):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


nb_name = os.path.basename(os.getcwd())  # notebook name
seed_torch(seed=42)

In [2]:
train

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,1.0,1.0,0.0,32.0,Winter,35.0,50.0,Fall,1.0,1.0
3956,ffa9794a,Winter,10,0,NaN,NaN,Spring,18.764678,53.5,76.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,0.0,NaN
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,0.0,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0
3958,ffed1dd5,Spring,13,0,Spring,70.0,Winter,12.235895,70.7,87.0,...,1.0,1.0,1.0,19.0,Spring,33.0,47.0,Spring,1.0,0.0


### テーブルデータセット

In [3]:
add_features = [
    "BMI_Age",
    "Internet_Hours_Age",
    "BMI_Internet_Hours",
    "BFP_BMI",
    "FFMI_BFP",
    "FMI_BFP",
    "LST_TBW",
    "BFP_BMR",
    "BFP_DEE",
    "BMR_Weight",
    "DEE_Weight",
    "SMM_Height",
    "Muscle_to_Fat",
    "Hydration_Status",
    "ICW_TBW",
]


def feature_engineering(df):
    # season_cols = [col for col in df.columns if "Season" in col]
    # df = df.drop(season_cols, axis=1)
    df["BMI_Age"] = df["Physical-BMI"] * df["Basic_Demos-Age"]
    df["Internet_Hours_Age"] = (
        df["PreInt_EduHx-computerinternet_hoursday"] * df["Basic_Demos-Age"]
    )
    df["BMI_Internet_Hours"] = (
        df["Physical-BMI"] * df["PreInt_EduHx-computerinternet_hoursday"]
    )
    df["BFP_BMI"] = df["BIA-BIA_Fat"] / df["BIA-BIA_BMI"]
    df["FFMI_BFP"] = df["BIA-BIA_FFMI"] / df["BIA-BIA_Fat"]
    df["FMI_BFP"] = df["BIA-BIA_FMI"] / df["BIA-BIA_Fat"]
    df["LST_TBW"] = df["BIA-BIA_LST"] / df["BIA-BIA_TBW"]
    df["BFP_BMR"] = df["BIA-BIA_Fat"] * df["BIA-BIA_BMR"]
    df["BFP_DEE"] = df["BIA-BIA_Fat"] * df["BIA-BIA_DEE"]
    df["BMR_Weight"] = df["BIA-BIA_BMR"] / df["Physical-Weight"]
    df["DEE_Weight"] = df["BIA-BIA_DEE"] / df["Physical-Weight"]
    df["SMM_Height"] = df["BIA-BIA_SMM"] / df["Physical-Height"]
    df["Muscle_to_Fat"] = df["BIA-BIA_SMM"] / df["BIA-BIA_FMI"]
    df["Hydration_Status"] = df["BIA-BIA_TBW"] / df["Physical-Weight"]
    df["ICW_TBW"] = df["BIA-BIA_ICW"] / df["BIA-BIA_TBW"]

    return df


train = feature_engineering(train)
train = train.replace([np.inf, -np.inf], np.nan)
for add_ in add_features:
    train[add_] = train[add_].fillna(0.0)
train = train.dropna(thresh=10, axis=0)

test = feature_engineering(test)
test = test.replace([np.inf, -np.inf], np.nan)
for add_ in add_features:
    test[add_] = test[add_].fillna(0.0)
test = test.dropna(thresh=10, axis=0)

In [4]:
# onehotEncoderの作成
from sklearn.preprocessing import OneHotEncoder


double_columns = [
    "FGC-FGC_SRR_Zone",
    "BIA-BIA_SMM",
    "Physical-Waist_Circumference",
    "BIA-BIA_FFMI",
    "FGC-FGC_CU",
    "PreInt_EduHx-computerinternet_hoursday",
    "BIA-BIA_ECW",
    "FGC-FGC_CU_Zone",
    "FGC-FGC_SRL_Zone",
    "BIA-BIA_DEE",
    "Physical-Weight",
    "Fitness_Endurance-Time_Mins",
    "FGC-FGC_SRR",
    "SDS-SDS_Total_T",
    "FGC-FGC_PU",
    "BIA-BIA_FFM",
    "FGC-FGC_TL_Zone",
    "Physical-BMI",
    "Physical-Systolic_BP",
    "Physical-HeartRate",
    "BIA-BIA_ICW",
    "Physical-Height",
    "FGC-FGC_SRL",
    "BIA-BIA_BMC",
    "Fitness_Endurance-Time_Sec",
    "BIA-BIA_Frame_num",
    "Basic_Demos-Age",
    "FGC-FGC_GSND_Zone",
    "Basic_Demos-Sex",
    "FGC-FGC_GSND",
    "BIA-BIA_LST",
    "FGC-FGC_TL",
    "BIA-BIA_BMI",
    "BIA-BIA_FMI",
    "PAQ_C-PAQ_C_Total",
    "BIA-BIA_Activity_Level_num",
    "FGC-FGC_GSD",
    "BIA-BIA_BMR",
    "BIA-BIA_Fat",
    "SDS-SDS_Total_Raw",
    "CGAS-CGAS_Score",
    "FGC-FGC_PU_Zone",
    "BIA-BIA_LDM",
    "Fitness_Endurance-Max_Stage",
    "PAQ_A-PAQ_A_Total",
    "BIA-BIA_TBW",
    "FGC-FGC_GSD_Zone",
    "Physical-Diastolic_BP",
]
from sklearn.preprocessing import StandardScaler


def create_dataset_(df, scaler=None, train=True):

    if scaler is None:
        print("create new scaler")
        scaler = StandardScaler()
        scaler.fit(df[double_columns + add_features])
        with open("./assets/scaler.pkl", "wb") as f:
            pickle.dump(scaler, f)

    double_feature = scaler.transform(df[double_columns + add_features])
    # 欠損値の補完
    double_feature = np.nan_to_num(double_feature)

    ids = df["id"].values.reshape(-1, 1)
    X = double_feature

    # DataFrameの作成
    ids_df = pd.DataFrame(ids, columns=["id"])
    X_df = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])

    if train:
        y = df["sii"].fillna(-1).values.reshape(-1, 1)
        y_df = pd.DataFrame(y, columns=["sii"])
        df = pd.concat([ids_df, X_df, y_df], axis=1)
    else:
        df = pd.concat([ids_df, X_df], axis=1)
    return df, scaler


train, scaler = create_dataset_(train)
test = create_dataset_(test, scaler=scaler, train=False)[0]

create new scaler


In [5]:
feature_imputer = KNNImputer(n_neighbors=5)
sii_imputer = KNNImputer(n_neighbors=5)

numeric_cols = test.select_dtypes(include=["float64", "int64"]).columns
numeric_feature_cols = numeric_cols.copy()
# numeric_feature_cols = numeric_feature_cols.drop("sii")

numeric_sii_cols = train.select_dtypes(include=["float64", "int64"]).columns

sii_inputed = sii_imputer.fit_transform(train[numeric_sii_cols])
feature_imputer.fit(test[numeric_feature_cols])
feature_inputed = feature_imputer.fit_transform(train[numeric_feature_cols])

train_imputed = pd.DataFrame(feature_inputed, columns=numeric_feature_cols)

for col in train.columns:
    if col not in numeric_cols:
        train_imputed[col] = train[col]

train_imputed["sii"] = train["sii"]
train = train_imputed

# train = train[train["sii"] > -1].reset_index(drop=True)
train = train[train["sii"].notnull()].reset_index(drop=True)

# sii_impute = pd.DataFrame(sii_inputed, columns=numeric_sii_cols)
# sii_impute["sii"] = sii_impute["sii"].round().astype(int)
# train["sii"] = sii_impute["sii"]

with open("feature_imputer.pkl", "wb") as f:
    pickle.dump(feature_imputer, f)

train.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,id,sii
0,-1.277596,-0.176702,0.000000,-0.209436,-0.953788,1.771623,-0.171600,-0.953742,-1.274301,-0.201970,...,0.015691,0.015630,0.531031,0.540327,0.113954,0.083339,0.251650,1.296492,00008ff9,2.0
1,0.782720,-0.225858,-0.948658,-0.380787,-0.699663,-0.968830,-0.202128,-0.953742,0.784744,-0.199625,...,0.015270,0.015181,0.629987,0.643793,0.021863,0.304658,0.209698,1.388511,000fd460,0.0
2,0.782720,0.000000,0.000000,0.000000,0.740376,0.858138,0.000000,1.048502,0.784744,0.000000,...,0.014948,0.014837,-0.374560,-0.406553,-0.263102,-0.139659,-0.235959,-0.993169,00105258,0.0
3,-1.277596,-0.094130,0.000000,-0.165178,0.570959,-0.968830,-0.071440,1.048502,-1.274301,-0.049816,...,0.016791,0.016926,0.309486,0.353385,0.156596,0.079541,0.191155,1.031686,00115b9f,1.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.014948,0.014837,-0.374560,-0.406553,-0.263102,-0.139659,-0.235959,-0.993169,0016bb22,-1.0


In [6]:
train_df = train

In [7]:
def read_parquet(base_dir, id_):
    path = os.path.join(base_dir, f"id={id_}", "part-0.parquet")
    return pd.read_parquet(path)


def get_valid_ids(base_dir):
    return [f.split("=")[1].split(".")[0] for f in os.listdir(base_dir)]


p = read_parquet(base_dir="../../inputs/series_train.parquet/", id_="ffcd4dbd")
# p = read_parquet(base_dir="../../inputs/series_train.parquet/", id_="10e46254")
# p

In [8]:
from glob import glob

# len(glob("../../normalized/*"))
len(glob("../../inputs/series_train.parquet/*"))

996

## Metric

In [9]:
from sklearn.metrics import *


def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights="quadratic")


def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(
        oof_non_rounded < thresholds[0],
        0,
        np.where(
            oof_non_rounded < thresholds[1],
            1,
            np.where(oof_non_rounded < thresholds[2], 2, 3),
        ),
    )


def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

## Model, Dataset

In [10]:
train_df

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,id,sii
0,-1.277596,-0.176702,0.000000,-0.209436,-0.953788,1.771623,-0.171600,-0.953742,-1.274301,-0.201970,...,0.015691,0.015630,0.531031,0.540327,0.113954,0.083339,0.251650,1.296492,00008ff9,2.0
1,0.782720,-0.225858,-0.948658,-0.380787,-0.699663,-0.968830,-0.202128,-0.953742,0.784744,-0.199625,...,0.015270,0.015181,0.629987,0.643793,0.021863,0.304658,0.209698,1.388511,000fd460,0.0
2,0.782720,0.000000,0.000000,0.000000,0.740376,0.858138,0.000000,1.048502,0.784744,0.000000,...,0.014948,0.014837,-0.374560,-0.406553,-0.263102,-0.139659,-0.235959,-0.993169,00105258,0.0
3,-1.277596,-0.094130,0.000000,-0.165178,0.570959,-0.968830,-0.071440,1.048502,-1.274301,-0.049816,...,0.016791,0.016926,0.309486,0.353385,0.156596,0.079541,0.191155,1.031686,00115b9f,1.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.014948,0.014837,-0.374560,-0.406553,-0.263102,-0.139659,-0.235959,-0.993169,0016bb22,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,0.782720,-0.054867,0.000000,-0.167095,0.401543,-0.055346,-0.018625,-0.953742,0.784744,-0.004582,...,0.016158,0.016209,0.348018,0.396190,0.181123,0.312400,0.249864,0.941730,ff8a2de4,1.0
3956,-1.277596,0.000000,-0.050054,0.000000,-0.953788,-0.968830,0.000000,-0.953742,-1.274301,0.000000,...,0.014948,0.014837,-0.374560,-0.406553,-0.263102,-0.139659,-0.235959,-0.993169,ffa9794a,-1.0
3957,0.782720,-0.066765,0.000000,-0.039070,0.316835,-0.968830,-0.004681,1.048502,0.784744,-0.020703,...,0.018636,0.018772,0.188757,0.182448,0.162631,0.011772,0.139885,0.918073,ffcd4dbd,1.0
3958,0.000000,0.131436,0.000000,-0.338825,0.000000,-0.055346,0.077913,0.000000,0.000000,0.319314,...,0.014225,0.013825,0.427455,0.694085,0.307287,-2.049356,0.355443,0.873188,ffed1dd5,0.0


In [11]:
from sklearn.model_selection import train_test_split

use_ids = list(
    train_df[train_df["sii"] != -1]["id"].unique()
)  # get_valid_ids(base_dir="../../normalized/")

len(use_ids)

2736

## Training

In [12]:
from tqdm import tqdm
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
use_ids = np.array(use_ids)
for train_index, valid_index in kf.split(use_ids):
    train_ids = [use_ids[i] for i in train_index]
    valid_ids = [use_ids[i] for i in valid_index]

    train_dataset = CMIDataset(
        table_df=train_df,
        valid_ids=use_ids,
        base_dir="../../inputs/series_train.parquet/",
        save_filename=nb_name,
    )

In [13]:
train_dataset[0]["time_input"].shape

torch.Size([31, 17280, 15])

In [14]:
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

CV = []

batch_size = 1

# use_ids = np.array(use_ids[:30]) # debug
use_ids = np.array(use_ids)

extract_df = train[train["id"].isin(use_ids)].reset_index(drop=True)

test_df = train[["id", "sii"]].copy()
# test_df["pred_sii"] = 0
oof_preds = []

for fold, (train_ids, valid_ids) in enumerate(
    skf.split(extract_df["id"], extract_df["sii"])
):
    print(f"################### fold:{fold} ###################")
    best_valid_score = -100

    train_ids = use_ids[train_ids]
    valid_ids = use_ids[valid_ids]

    train_dataset = CMIDataset(
        table_df=train_df,
        valid_ids=train_ids,
        base_dir="../../inputs/series_train.parquet/",
        save_filename=nb_name,
    )
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=30
    )

    vlaid_dataset = CMIDataset(
        table_df=train_df,
        valid_ids=valid_ids,
        base_dir="../../inputs/series_train.parquet/",
        save_filename=nb_name,
    )

    valid_loader = DataLoader(
        vlaid_dataset, batch_size=batch_size, shuffle=False, num_workers=30
    )
    # data_loader = DataLoader(dataset, batch_size=1, shuffle=True)

    # model = TimeEncoder(input_size=26, hidden_size=13, num_layers=2).to("cuda")
    model = CMIModel(input_size=26, hidden_size=13, num_layers=2).to("cuda")

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    spot_oof_preds = []

    for epoch in range(5):
        total_train_loss = 0
        total_valid_loss = 0

        train_pred = []
        valid_pred = []
        trian_gt = []
        valid_gt = []

        tq = tqdm(train_loader)
        for i, data in enumerate(train_loader):
            table_input = data["table_input"].to("cuda")
            time_input = data["time_input"].to("cuda")
            mask = data["mask"].to("cuda").to(torch.float32)
            target_ = data["output"].to("cuda")
            optimizer.zero_grad()
            output, attention_weight = model(table_input, time_input, active_mask=mask)
            loss = criterion(output, target_)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

            train_pred.append(output.detach().cpu().numpy())
            trian_gt.append(target_.detach().cpu().numpy())

            tq.set_postfix(loss=total_train_loss / (i + 1))
            tq.update()
        tq.close()

        tq = tqdm(valid_loader)
        for i, data in enumerate(valid_loader):
            table_input = data["table_input"].to("cuda")
            time_input = data["time_input"].to("cuda")
            mask = data["mask"].to("cuda").to(torch.float32)
            target_ = data["output"].to("cuda")
            output, attention_weight = model(table_input, time_input, active_mask=mask)
            loss = criterion(output, target_)
            total_valid_loss += loss.item()

            valid_pred.append(output.detach().cpu().numpy())
            valid_gt.append(target_.detach().cpu().numpy())

            tq.set_postfix(loss=total_valid_loss / (i + 1))
            tq.update()
        tq.close()

        metric_train_pred = np.concatenate(train_pred)
        metric_valid_pred = np.concatenate(valid_pred)
        metric_train_gt = np.concatenate(trian_gt)
        metric_valid_gt = np.concatenate(valid_gt)

        train_score = quadratic_weighted_kappa(
            metric_train_gt, metric_train_pred.round(0).astype(int)
        )

        valid_score = quadratic_weighted_kappa(
            metric_valid_gt, metric_valid_pred.round(0).astype(int)
        )

        print(
            f"epoch: {epoch}, loss: {total_train_loss / len(train_loader)}, valid_loss: {total_valid_loss / len(valid_loader)}, train_score: {train_score}, valid_score: {valid_score}"
        )

        if valid_score > best_valid_score:
            best_valid_score = valid_score
            torch.save(model.state_dict(), f"./assets/model_{fold}.pth")

            spot_oof_preds = []
            for i, id_ in enumerate(valid_ids):
                spot_oof_preds.append({"id": id_, "pred_sii": valid_pred[i][0][0]})

    oof_preds.append(spot_oof_preds)
    CV.append(best_valid_score)

print(f"CV: {np.mean(CV)}")

################### fold:0 ###################


100%|██████████| 684/684 [01:25<00:00,  7.97it/s, loss=0.593]


epoch: 0, loss: 0.5261262106394098, valid_loss: 0.5930575921885844, train_score: 0.3202360702572151, valid_score: 0.2671737207833227


100%|██████████| 684/684 [00:17<00:00, 38.98it/s, loss=0.45] 


epoch: 1, loss: 0.5732467797716541, valid_loss: 0.4502671581064991, train_score: 0.34878755192724364, valid_score: 0.4033048794555312


...

epoch: 2, loss: 0.4500814243557907, valid_loss: 0.5323154221667593, train_score: 0.4238409666293854, valid_score: 0.38098731697030674


100%|██████████| 684/684 [00:17<00:00, 39.57it/s, loss=0.528]


epoch: 3, loss: 0.432815089524101, valid_loss: 0.5280419899957094, train_score: 0.4568822440161855, valid_score: 0.2914251103227482


100%|██████████| 684/684 [00:17<00:00, 39.86it/s, loss=0.622]

epoch: 4, loss: 0.41547917749576146, valid_loss: 0.621983830927687, train_score: 0.48828920570264767, valid_score: 0.3063709897300737
CV: 0.3925656292740731


In [15]:
oof_preds_df = pd.concat([pd.DataFrame(p) for p in oof_preds], axis=0).reset_index(
    drop=True
)
oof_preds_df.head()

,id,pred_sii
0,00105258,0.781073
1,00115b9f,0.318422
2,0038ba98,0.636342
3,00ae59c9,0.939089
4,00f574e9,1.211709


In [16]:
test_pred_df = test_df.merge(oof_preds_df, on="id", how="inner")
test_pred_df.head()

,id,sii,pred_sii
0,00008ff9,2.0,0.585018
1,000fd460,0.0,0.310536
2,00105258,0.0,0.781073
3,00115b9f,1.0,0.318422
4,001f3379,1.0,0.326469


In [17]:
from scipy.optimize import minimize

KappaOPtimizer = minimize(
    evaluate_predictions,
    x0=[0.5, 1.5, 2.5],
    args=(test_pred_df["sii"], test_pred_df["pred_sii"]),
    method="Nelder-Mead",
)
assert KappaOPtimizer.success, "Optimization did not converge."

oof_tuned = threshold_Rounder(test_pred_df["pred_sii"], KappaOPtimizer.x)
tKappa = quadratic_weighted_kappa(test_pred_df["sii"], oof_tuned)
print(f"tuned Kappa: {tKappa}")

tuned Kappa: 0.4432175021188638


In [18]:
print(KappaOPtimizer.x)

[0.53658205 1.04475487 2.7068056 ]


In [19]:
train

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,id,sii
0,-1.277596,-0.176702,0.000000,-0.209436,-0.953788,1.771623,-0.171600,-0.953742,-1.274301,-0.201970,...,0.015691,0.015630,0.531031,0.540327,0.113954,0.083339,0.251650,1.296492,00008ff9,2.0
1,0.782720,-0.225858,-0.948658,-0.380787,-0.699663,-0.968830,-0.202128,-0.953742,0.784744,-0.199625,...,0.015270,0.015181,0.629987,0.643793,0.021863,0.304658,0.209698,1.388511,000fd460,0.0
2,0.782720,0.000000,0.000000,0.000000,0.740376,0.858138,0.000000,1.048502,0.784744,0.000000,...,0.014948,0.014837,-0.374560,-0.406553,-0.263102,-0.139659,-0.235959,-0.993169,00105258,0.0
3,-1.277596,-0.094130,0.000000,-0.165178,0.570959,-0.968830,-0.071440,1.048502,-1.274301,-0.049816,...,0.016791,0.016926,0.309486,0.353385,0.156596,0.079541,0.191155,1.031686,00115b9f,1.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.014948,0.014837,-0.374560,-0.406553,-0.263102,-0.139659,-0.235959,-0.993169,0016bb22,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,0.782720,-0.054867,0.000000,-0.167095,0.401543,-0.055346,-0.018625,-0.953742,0.784744,-0.004582,...,0.016158,0.016209,0.348018,0.396190,0.181123,0.312400,0.249864,0.941730,ff8a2de4,1.0
3956,-1.277596,0.000000,-0.050054,0.000000,-0.953788,-0.968830,0.000000,-0.953742,-1.274301,0.000000,...,0.014948,0.014837,-0.374560,-0.406553,-0.263102,-0.139659,-0.235959,-0.993169,ffa9794a,-1.0
3957,0.782720,-0.066765,0.000000,-0.039070,0.316835,-0.968830,-0.004681,1.048502,0.784744,-0.020703,...,0.018636,0.018772,0.188757,0.182448,0.162631,0.011772,0.139885,0.918073,ffcd4dbd,1.0
3958,0.000000,0.131436,0.000000,-0.338825,0.000000,-0.055346,0.077913,0.000000,0.000000,0.319314,...,0.014225,0.013825,0.427455,0.694085,0.307287,-2.049356,0.355443,0.873188,ffed1dd5,0.0
